<a href="https://colab.research.google.com/github/HayyanWasi/openai-agent-learning/blob/main/04_tool_calling_agentSDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **Tool Calling**

tool calling are components of sdk that allow agents to perform actions such as web searching file searching interacting with other APIs or using a computer

# **BASIC AGENT STRUCTURE**

In [ ]:
!pip install -Uq openai-agents openai google-generativeai openai-agents python-dotenv

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel

from agents.run import RunConfig

from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

if not gemini_api_key:
  raise ValueError("GEMINI_API_KEY not found in userdata")

external_client = AsyncOpenAI(
    api_key = gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta",
)
model = OpenAIChatCompletionsModel(
    model = "gemini-2.0-flash",
    openai_client=external_client
)
config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=(True)
)

# **HOSTED TOOLS**
> They are built in functions
> **They are of 3 types**

1. WebSearchTool-allow agent to   perform  live web searching

2. FileSearchTool-for taking information from vector stores

3. ComputerTool-enable automation task on your computer


Tool name is name of function
descripton for agent is comes from docstring
sechema is automatically generated from arguments
description of input is also comes from docstring

In [10]:
from agents import function_tool

@function_tool
def get_weather(city:str)-> str:           #function name is taken as tool name
  print(f"getting weather for {city}")     #description comes from docstrings
  return f"the weather in {city} is sunny" #schema is generated from arguments


@function_tool
def get_temperature(city:str)->str:
  print(f"getting temperature for {city}")
  return "70 degrees"

agent = Agent(
      name="weather agent",
      instructions ="you are the local weather agent.YOur are given a city and you need to tell the weather and temperature.",
      tools=[get_weather, get_temperature], #this registers function as a tool.Agent will automaticlly decide when to call this tool based on user's query this is part of tool calling mechanism in agent SDK
      model = model
  )

result = await Runner.run(agent,"temp in karachi")
print(result.final_output)



getting temperature for karachi
The temperature in Karachi is 70 degrees.



# **WEB SEARCH TOOL**
used for web searching

For Hosting tool openai's api is required

In [ ]:
# from agents import WebSearchTool, Agent, Runner

# # Create an agent that uses the hosted WebSearchTool
# news_agent = Agent(
#     name="News Reporter",
#     instructions="You're a news reporter. Find recent news articles about Pakistan politics.",
#     tools=[WebSearchTool()],
#     model=model
# )

# # Run the agent with a query
# news_result = await Runner.run(news_agent, "Find recent news about Pakistan politics")
# print(news_result.final_output)
